In [1]:
import math
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

import matplotlib.pyplot as plt

In [2]:
deliveries = pd.read_csv('deliveries_updated_mens_ipl.csv')
matches = pd.read_csv('matches_updated_mens_ipl.csv')

In [3]:
df = deliveries.copy()

In [4]:
mdf = matches.copy()

In [5]:
df = df.rename(columns = {'matchId' : 'match_id'}) 
mdf = mdf.rename(columns = {'matchId' : 'match_id'})

In [6]:
req_df = pd.merge(df,mdf, on = 'match_id', how = 'left')

In [8]:
def phase(over):
    if over < 6:
        return 'Powerplay Overs'
    elif over < 17:
        return 'Middle Overs'
    else:
        return 'Death Overs'

In [9]:
def BPD(Balls, Dismissals):
    if Dismissals > 0:
        return Balls/Dismissals
    else :
        return Balls/1
    
def BPB(Balls, Boundaries):
    if Boundaries > 0:
        return Balls/Boundaries
    else:
        return Balls/1

In [10]:
def ByCustom(df,current_venue, current_phase, current_opposition):

    df = df[df.venue == current_venue]
    df = df[df.Phase == current_phase]
    df = df[df.bowling_team == current_opposition]
    df.reset_index(inplace = True, drop = True)
    
    df['isDot'] = df['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].sum()).reset_index().rename(columns = {'batsman_runs':'Runs'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id' : 'Balls'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'match_id' : 'Innings'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns = {'player_dismissed' : 'Dismissals'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour' : 'Fours'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix' : 'Sixes'})
    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot' : 'Dots'})
    
    df = pd.merge(innings,runs, on ='batsman').merge(balls,on='batsman').merge(dismissals,on='batsman').merge(fours,on='batsman').merge(sixes,on='batsman').merge(dots, on='batsman')
    
    df['RPI'] = df.apply(lambda x : (x['Runs']/x['Innings']), axis = 1)
    df['SR'] = df.apply(lambda x : 100*(x['Runs']/x['Balls']), axis = 1)
    df['BPD'] = df.apply(lambda x : BPD(x['Balls'], x['Dismissals']),axis = 1)
    df['BPB'] = df.apply(lambda x : BPB(x['Balls'], (x['Fours'] + x['Sixes'])),axis = 1)
    df['Dot%'] = df.apply(lambda x : (x['Dots']/x['Balls']), axis = 1)
    
    return df

In [11]:
req_df['Phase'] = req_df['over'].apply(lambda x : phase(x))

### Powerplay Overs

In [23]:
df2 = ByCustom(req_df,'MA Chidambaram Stadium', 'Powerplay Overs', 'Chennai Super Kings')

In [33]:
df2 = df2[df2.Balls >= 12]

In [14]:
wt_sr, wt_rpi, wt_bpd, wt_dot_percentage = 0.13, 0.27, 0.16, 0.45

### Normalizing

In [35]:
df2['calc_SR'] = df2['SR'].apply(lambda x: x*x) 
df2['calc_RPI'] = df2['RPI'].apply(lambda x: x*x) 
df2['calc_BPD'] = df2['BPD'].apply(lambda x: x*x) 
df2['calc_Dot%'] = df2['Dot%'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df2[['calc_SR','calc_RPI', 'calc_BPD', 'calc_Dot%']].sum(axis = 0))

df2['calc_SR'] = df2['calc_SR'].apply(lambda x: x/sq_sr) 
df2['calc_RPI'] = df2['calc_RPI'].apply(lambda x: x/sq_rpi) 
df2['calc_BPD'] = df2['calc_BPD'].apply(lambda x: x/sq_bpd) 
df2['calc_Dot%'] = df2['calc_Dot%'].apply(lambda x: x/sq_dot_percentage)

df2['calc_SR'] = df2['calc_SR'].apply(lambda x: x*wt_sr) 
df2['calc_RPI'] = df2['calc_RPI'].apply(lambda x: x*wt_rpi) 
df2['calc_BPD'] = df2['calc_BPD'].apply(lambda x: x*wt_bpd) 
df2['calc_Dot%'] = df2['calc_Dot%'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df2['calc_SR']), min(df2['calc_SR'])
best_rpi, worst_rpi = max(df2['calc_RPI']), min(df2['calc_RPI'])
best_bpd, worst_bpd = max(df2['calc_BPD']), min(df2['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df2['calc_Dot%']), max(df2['calc_Dot%'])

### Comparison

In [34]:
df2['dev_best_SR'] = df2['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
df2['dev_best_RPI'] = df2['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
df2['dev_best_BPD'] = df2['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
df2['dev_best_dot_percentage'] = df2['calc_Dot%'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df2['dev_best_sqrt'] = df2.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1) 

df2['dev_worst_SR'] = df2['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
df2['dev_worst_RPI'] = df2['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
df2['dev_worst_BPD'] = df2['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
df2['dev_worst_dot_percentage'] = df2['calc_Dot%'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df2['dev_worst_sqrt'] = df2.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1) 

In [36]:
df2['Score'] = df2.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [37]:
PP_df = df2[[ 'batsman','Innings', 'Runs', 'Balls', 'Dismissals', 'Dot%', 'Score']].sort_values(['Score'], ascending = False).reset_index(drop = True)

In [38]:
PP_df

,batsman,Innings,Runs,Balls,Dismissals,Dot%,Score
0,SS Iyer,1,35,17,0,0.117647,0.965559
1,MK Pandey,1,27,12,0,0.416667,0.961918
2,RV Uthappa,2,39,23,1,0.478261,0.566303
3,KL Rahul,1,26,16,0,0.250000,0.549653
4,RA Tripathi,1,18,12,0,0.416667,0.335119
5,S Dhawan,1,19,13,1,0.384615,0.309886
6,SA Yadav,1,17,13,0,0.384615,0.184918
7,RG Sharma,2,21,16,1,0.312500,0.169852
8,DA Warner,1,25,24,0,0.500000,0.148433
9,CA Lynn,2,22,21,2,0.523810,0.060076


### Middle Overs

In [12]:
df3 = ByCustom(req_df,'MA Chidambaram Stadium', 'Middle Overs', 'Chennai Super Kings')

In [15]:
df3 = df3[df3.Balls >= 15 ]

### Normalizing the Values

In [16]:
df3['calc_SR'] = df3['SR'].apply(lambda x: x*x) 
df3['calc_RPI'] = df3['RPI'].apply(lambda x: x*x) 
df3['calc_BPD'] = df3['BPD'].apply(lambda x: x*x) 
df3['calc_Dot%'] = df3['Dot%'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df3[['calc_SR','calc_RPI', 'calc_BPD', 'calc_Dot%']].sum(axis = 0))

df3['calc_SR'] = df3['calc_SR'].apply(lambda x: x/sq_sr) 
df3['calc_RPI'] = df3['calc_RPI'].apply(lambda x: x/sq_rpi) 
df3['calc_BPD'] = df3['calc_BPD'].apply(lambda x: x/sq_bpd) 
df3['calc_Dot%'] = df3['calc_Dot%'].apply(lambda x: x/sq_dot_percentage)

df3['calc_SR'] = df3['calc_SR'].apply(lambda x: x*wt_sr) 
df3['calc_RPI'] = df3['calc_RPI'].apply(lambda x: x*wt_rpi) 
df3['calc_BPD'] = df3['calc_BPD'].apply(lambda x: x*wt_bpd) 
df3['calc_Dot%'] = df3['calc_Dot%'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df3['calc_SR']), min(df3['calc_SR'])
best_rpi, worst_rpi = max(df3['calc_RPI']), min(df3['calc_RPI'])
best_bpd, worst_bpd = max(df3['calc_BPD']), min(df3['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df3['calc_Dot%']), max(df3['calc_Dot%'])

### Comparison

In [17]:
df3['dev_best_SR'] = df3['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
df3['dev_best_RPI'] = df3['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
df3['dev_best_BPD'] = df3['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
df3['dev_best_dot_percentage'] = df3['calc_Dot%'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df3['dev_best_sqrt'] = df3.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1) 

df3['dev_worst_SR'] = df3['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
df3['dev_worst_RPI'] = df3['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
df3['dev_worst_BPD'] = df3['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
df3['dev_worst_dot_percentage'] = df3['calc_Dot%'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df3['dev_worst_sqrt'] = df3.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1) 

In [18]:
df3['Score'] = df3.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [21]:
middle_df = df3[[ 'batsman','Innings', 'Runs', 'Balls', 'Dismissals', 'Dot%', 'Score']].sort_values(['Score'], ascending = False).reset_index(drop = True)

In [39]:
middle_df

,batsman,Innings,Runs,Balls,Dismissals,Dot%,Score
0,RG Sharma,1,50,34,1,0.352941,0.909028
1,MK Pandey,1,48,31,0,0.161290,0.898914
2,SA Yadav,1,49,34,0,0.235294,0.896013
3,AD Russell,2,70,54,0,0.444444,0.765921
4,DA Warner,1,32,23,1,0.173913,0.568237
5,BA Stokes,1,26,18,0,0.111111,0.519215
6,SN Khan,1,40,38,0,0.342105,0.505009
7,KD Karthik,2,36,31,1,0.387097,0.247744
8,E Lewis,1,20,18,1,0.333333,0.181738
9,KL Rahul,1,28,29,0,0.206897,0.171890


### Death Overs

In [47]:
df4 = ByCustom(req_df,'MA Chidambaram Stadium', 'Death Overs', 'Chennai Super Kings')

In [53]:
df4 = df4[df4.Balls >= 6]

### Normalizing

In [54]:
df4['calc_SR'] = df4['SR'].apply(lambda x: x*x) 
df4['calc_RPI'] = df4['RPI'].apply(lambda x: x*x) 
df4['calc_BPD'] = df4['BPD'].apply(lambda x: x*x) 
df4['calc_Dot%'] = df4['Dot%'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df4[['calc_SR','calc_RPI', 'calc_BPD', 'calc_Dot%']].sum(axis = 0))

df4['calc_SR'] = df4['calc_SR'].apply(lambda x: x/sq_sr) 
df4['calc_RPI'] = df4['calc_RPI'].apply(lambda x: x/sq_rpi) 
df4['calc_BPD'] = df4['calc_BPD'].apply(lambda x: x/sq_bpd) 
df4['calc_Dot%'] = df4['calc_Dot%'].apply(lambda x: x/sq_dot_percentage)

df4['calc_SR'] = df4['calc_SR'].apply(lambda x: x*wt_sr) 
df4['calc_RPI'] = df4['calc_RPI'].apply(lambda x: x*wt_rpi) 
df4['calc_BPD'] = df4['calc_BPD'].apply(lambda x: x*wt_bpd) 
df4['calc_Dot%'] = df4['calc_Dot%'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df4['calc_SR']), min(df4['calc_SR'])
best_rpi, worst_rpi = max(df4['calc_RPI']), min(df4['calc_RPI'])
best_bpd, worst_bpd = max(df4['calc_BPD']), min(df4['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df4['calc_Dot%']), max(df4['calc_Dot%'])

### Comparison

In [55]:
df4['dev_best_SR'] = df4['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
df4['dev_best_RPI'] = df4['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
df4['dev_best_BPD'] = df4['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
df4['dev_best_dot_percentage'] = df4['calc_Dot%'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df4['dev_best_sqrt'] = df4.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1) 

df4['dev_worst_SR'] = df4['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
df4['dev_worst_RPI'] = df4['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
df4['dev_worst_BPD'] = df4['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
df4['dev_worst_dot_percentage'] = df4['calc_Dot%'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df4['dev_worst_sqrt'] = df4.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1) 

In [56]:
df4['Score'] = df4.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [57]:
death_df = df4[[ 'batsman','Innings', 'Runs', 'Balls', 'Dismissals', 'Dot%', 'Score']].sort_values(['Score'], ascending = False).reset_index(drop = True)

In [58]:
death_df

,batsman,Innings,Runs,Balls,Dismissals,Dot%,Score
0,AD Russell,2,68,28,0,0.357143,0.996618
1,BA Stokes,1,20,8,1,0.250000,0.889182
2,JC Archer,1,13,7,0,0.142857,0.384121
3,KA Pollard,1,13,8,0,0.250000,0.195279
4,HH Pandya,2,21,13,0,0.230769,0.186825
5,V Shankar,1,10,7,1,0.428571,0.088092
6,SN Khan,1,13,11,1,0.272727,0.029816
7,MK Pandey,1,8,7,0,0.000000,0.020066
8,SA Yadav,1,5,7,0,0.714286,0.000000


### Conclusion :
### From the three dataframes here (PP_df, middle_df, death_df), it can be seen that Manish Pandey has been one of the players who has been prolific in the Powerplay overs and in the middle overs. But when it comes to death overs, it is the Caribbean power Andre Russell who is mindblowing without being dismissed in the death overs. There are also other players like Surya, Rohit and Warner who have been good in the Powerplay and in the middle overs.